Homework Assignment 1: Analyzing Sales Data

In [1]:
import pandas as pd

In [3]:
df=pd.read_csv('C:\python\databases\sales_data.csv')
df.head()

,Date,Product,Category,Quantity,Price
0,2023-01-01,Laptop,Electronics,10,800
1,2023-01-01,T-Shirt,Clothing,5,20
2,2023-01-02,Smartphone,Electronics,8,400
3,2023-01-02,Coffee Maker,Home,12,50
4,2023-01-03,Jeans,Clothing,15,30


Tasks:

1. Group the data by the Category column and calculate the following aggregate statistics for each category:
    -Total quantity sold.
    -Average price per unit.
    -Maximum quantity sold in a single transaction.
2. Identify the top-selling product in each category based on the total quantity sold.
3. Find the date on which the highest total sales (quantity * price) occurred.

In [ ]:
#1.1 Total quantity sold.
df.groupby('Category')['Quantity'].agg('sum')

Category
Clothing       157
Electronics    183
Home           144
Name: Quantity, dtype: int64

In [22]:
#1.2 Average price per unit.

df.groupby('Category').agg({'Price': 'mean'})

,Price
Category,
Clothing,31.176471
Electronics,276.764706
Home,55.000000


In [ ]:
#1.3 Maximum quantity sold in a single transaction.

df.groupby('Category')['Quantity'].agg('max')

Category
Clothing       15
Electronics    15
Home           14
Name: Quantity, dtype: int64

In [34]:
#2. Identify the top-selling product in each category based on the total quantity sold.

df.loc[df.groupby('Category')['Quantity'].idxmax(), ['Category', 'Product', 'Quantity']]

,Category,Product,Quantity
4,Clothing,Jeans,15
12,Electronics,Smart TV,15
44,Home,Pressure Cooker,14


In [38]:
#3. Find the date on which the highest total sales (quantity * price) occurred.

df['TotalSales']=df['Quantity']*df['Price']
df.groupby('Date')['TotalSales'].agg('sum').idxmax()


'2023-01-07'

Homework Assignment 2: Examining Customer Orders

In [6]:
df=pd.read_csv(r'C:\python\databases\customer_orders.csv')
df

,OrderID,CustomerID,Product,Quantity,Price
0,1,101,Laptop,2,800
1,2,102,Headphones,1,150
2,3,103,Smartphone,3,400
3,4,101,External Hard Drive,2,80
4,5,102,Backpack,1,40
...,...,...,...,...,...
95,96,103,Blender,1,60
96,97,104,Sport Shoes,2,40
97,98,105,Comforter Set,1,45
98,99,101,Wireless Earbuds,2,120


Tasks:

1. Group the data by CustomerID and filter out customers who have made less than 20 orders.
2. Identify customers who have ordered products with an average price per unit greater than $120.
3. Find the total quantity and total price for each product ordered, and filter out products that have a total quantity less than 5 units.

In [55]:
#1
a=df.groupby('CustomerID').agg({'OrderID': 'count'}).rename(columns={'OrderID': 'count_orders'})

valid_customers=a[a['count_orders']<20].index  
filtered=df[df['CustomerID'].isin(valid_customers)]


In [ ]:
#2
a=df.groupby('Product').agg({'Price': 'mean'}).rename(columns={'Price': 'avg_price'})
high_price_products=a[a['avg_price'] > 120].index 

filtered_customers=df[df['Product'].isin(high_price_products)]['CustomerID'].unique()

pd.DataFrame({'CustomerID': filtered_customers})



,CustomerID
0,101
1,102
2,103
3,104
4,105


In [8]:
#3

a=df.groupby('Product').agg({'Quantity': 'sum', 'Price': 'sum'}).rename(columns={'Quantity': 'quantity', 'Price': 'price'})
filtered=a[a['quantity']<5]

Homework Assignment 3: Population Salary Analysis

Homework Assignment 3: Population Salary Analysis

1. "task\population.db" sqlite database has population table.
2. "task\population salary analysis.xlsx" file defines Salary Band categories.
    -Read the data from population table and calculate following measures:
    -Percentage of population for each salary category;
    -Average salary in each salary category;
    -Median salary in each salary category;
    -Number of population in each salary category;
3. Calculate the same measures in each State

In [1]:
#1
import sqlite3 
import pandas as pd

In [2]:
conn=sqlite3.connect('C:\python\python_homeworks2\population.db')
cur=conn.cursor()
query=('select *from population')
cur.execute(query)

a=cur.fetchall()
conn.close()


In [20]:
df=pd.DataFrame(a)
df=df.rename(columns={0: 'id', 1: 'first_name', 2: 'last_name', 3: 'email', 4: 'gender', 5: 'salary', 6: 'state'})
df

,id,first_name,last_name,email,gender,salary,state
0,1,Armin,Coltart,acoltart0@abc.net.au,Male,368693,District of Columbia
1,2,Mia,Tuddenham,mtuddenham1@addthis.com,Female,154398,Florida
2,3,Kirsteni,Brafield,kbrafield2@arizona.edu,Female,1230304,Georgia
3,4,Phylis,Furlong,pfurlong3@merriam-webster.com,Female,1567795,California
4,5,Wandis,Loveredge,wloveredge4@hatena.ne.jp,Female,1136950,Alabama
...,...,...,...,...,...,...,...
11646,11647,Nell,Perez,nperezhy@flavors.me,Female,1909496,Texas
11647,11648,Ugo,Pethrick,upethrickhz@discuz.net,Male,830390,New Mexico
11648,11649,Hartwell,Hadigate,hhadigatei0@cam.ac.uk,Male,24363,Washington
11649,11650,Avrit,Bastin,abastini1@symantec.com,Female,1869988,Rhode Island


2. Read the data from population table and calculate following measures:
    -Percentage of population for each salary category;
    -Average salary in each salary category;
    -Median salary in each salary category;
    -Number of population in each salary category;

In [28]:
bins=[0, 50000, 200000, 500000, 1000000, float('inf')]
labels=["Low", "Medium", "High", "Very High", "Top"]
df["salary_category"] = pd.cut(df["salary"], bins=bins, labels=labels)

salary_stats = df.groupby("salary_category")["salary"].agg([
    ("Count", "count"),
    ("Percentage", lambda x: len(x) / len(df) * 100),
    ("Average Salary", "mean"),
    ("Median Salary", "median")])

salary_stats

C:\Users\sarva\AppData\Local\Temp\ipykernel_4308\4217807486.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  salary_stats = df.groupby("salary_category")["salary"].agg([


,Count,Percentage,Average Salary,Median Salary
salary_category,,,,
Low,280,2.403227,2.332373e+04,23082.0
Medium,871,7.475753,1.237029e+05,123260.0
High,1805,15.492232,3.529727e+05,353020.0
Very High,2930,25.148056,7.498364e+05,749286.0
Top,5765,49.480731,1.494657e+06,1491921.0


In [29]:
#3. Calculate the same measures in each State

bins=[0, 50000, 200000, 500000, 1000000, float('inf')]
labels=["Low", "Medium", "High", "Very High", "Top"]
df["salary_category"] = pd.cut(df["salary"], bins=bins, labels=labels)

state_salary_stats = df.groupby(["state", "salary_category"])["salary"].agg([
    ("Count", "count"),
    ("Percentage", lambda x: len(x) / len(df) * 100),
    ("Average Salary", "mean"),
    ("Median Salary", "median")
]).reset_index()

state_salary_stats

C:\Users\sarva\AppData\Local\Temp\ipykernel_4308\3725182865.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  state_salary_stats = df.groupby(["state", "salary_category"])["salary"].agg([


,state,salary_category,Count,Percentage,Average Salary,Median Salary
0,Alabama,Low,2,0.017166,2.580350e+04,25803.5
1,Alabama,Medium,20,0.171659,1.332225e+05,136273.0
2,Alabama,High,39,0.334735,3.446262e+05,359204.0
3,Alabama,Very High,64,0.549309,7.573528e+05,740502.5
4,Alabama,Top,121,1.038537,1.448274e+06,1449078.0
...,...,...,...,...,...,...
250,Wyoming,Low,1,0.008583,2.772800e+04,27728.0
251,Wyoming,Medium,0,NaN,NaN,NaN
252,Wyoming,High,0,NaN,NaN,NaN
253,Wyoming,Very High,1,0.008583,9.703660e+05,970366.0
